# Using CBFs for forbidden zones

To deal with the forbidden zones we can utilize a CBF with half plane constraints
and solve it with quadratic programming.

$$
\begin{aligned}
U_1 &= \{a_1^\top u + b_1 \leq 0\} \\
U_2 &= \{a_2^\top u + b_2 \leq 0\} \\
U_b &= U_{b,1} \cap U_{b,2}
\end{aligned}
$$

Which can than be formulated into

$$
\begin{aligned}
\min_{\substack{u \\\kappa_i \in U_{b,i}}} & \quad u^\top H u + p \delta \\
\text{s.t.} & \quad I u = \kappa \\
            & \quad u_\text{min} \leq u \leq u_\text{max}
\end{aligned}
$$